In [52]:
from lookup import LookupCreator
from sensepolar.polarity import WordPolarity
from sensepolar.bertEmbed import BERTWordEmbeddings
from sensepolar.polarDim import PolarDimensions
from sensepolar.oracle.dictionaryapi import Dictionary

In [5]:
# !pip install -U -q ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !pip install plotly
# !pip install chart-studio
# !pip install cufflinks

In [108]:
out_path = './antonyms/'

dimensions = [['hot', 'cold'], ['bad', 'good'], ['intelligent', 'unintelligent'], ['capable', 'incapable'], ['red', 'green']]

# dictionary = Dictionary('wordnet', api_key='')    
# dictionary = Dictionary('dictionaryapi', api_key='b4b51989-1b9d-4690-8975-4a83df13efc4 ')
dictionary = Dictionary('wordnik', api_key='6488daf20061aa3e6200c013b470fa8ef1f2678c19b36ef05')
lookupSpace = LookupCreator(dictionary, dimensions, out_path)
lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = BERTWordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)


wp = WordPolarity(model, antonym_path=antonym_path, method='base-change', number_polar=4)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Please enter at least one example sentence for the word: bad
Unable to create POLAR dimensions.
Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl


In [54]:
word = "sun"
context = "The sun is shining today."
polarity_base_change = wp.analyze_word(word, context)
print("Word polarity using base change method: ", polarity_base_change)

Top: 1
Dimension: hot <------> cold
['having or showing great eagerness or enthusiasm', 'newly made', 'characterized by violent and forceful activity or movement; very intense', 'newest or most recent', 'very good; often used in the negative', 'sexually excited or exciting', 'very unpleasant or even dangerous', 'producing a burning sensation on the taste nerves', 'wanted by the police', 'extended meanings; especially of psychological heat; marked by intensity or vehemence especially of passion or enthusiasm', 'having or dealing with dangerously high levels of radioactivity', '(color) bold and intense', 'charged or energized with electricity', 'having or bringing unusually good luck', 'performed or performing with unusually great skill and daring and energy', 'marked by excited activity', 'of a seeker; very near to the object sought', 'very popular or successful', 'used of physical heat; having a high or higher than desirable temperature or giving off heat or feeling or causing a sensat

In [55]:
import plotly.graph_objects as go

fig = go.Figure(data = go.Scatterpolar(r=[0.3378, 0.1248, 0.10427, 0.09925], theta=["hot", "intelligent", "capable", "bad"]))

fig.update_traces(fill="toself")
fig.update_layout(
    polar=dict(
    radialaxis_angle = -45,
    angularaxis = dict(
        direction ="clockwise",
        period= 4
        )
    )
)

fig.show()

In [32]:
wp = WordPolarity(model, antonym_path=antonym_path, method='projection', number_polar=4)
polarity_projection = wp.analyze_word(word, context)
print("Word polarity using projection method: ", polarity_projection)

Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Top: 1
Dimension: hot <------> cold
['having or showing great eagerness or enthusiasm', 'newly made', 'characterized by violent and forceful activity or movement; very intense', 'newest or most recent', 'very good; often used in the negative', 'sexually excited or exciting', 'very unpleasant or even dangerous', 'producing a burning sensation on the taste nerves', 'wanted by the police', 'extended meanings; especially of psychological heat; marked by intensity or vehemence especially of passion or enthusiasm', 'having or dealing with dangerously high levels of radioactivity', '(color) bold and intense', 'charged or energized with electricity', 'having or bringing unusually good luck', 'performed or performing with unusually great skill and daring and energy', 'marked by excited activity', 'of a seeker; very near to the object sought', 'very popular or successful', 'used of physical heat; having a high or higher than desirable

In [33]:
import chart_studio as plotly
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import pandas as pd

init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


import os
from IPython.display import Image, display, HTML



In [106]:
import plotly.graph_objects as go
import numpy as np
from collections import defaultdict


def plot_word_polarity(words, polar_dimension):
    # create dictionary with antonyms as key and (word,polar) as value
    antonym_dict = defaultdict(list)
    for w_i in range(len(words)):
        for antonym1, antonym2, value in polar_dimension[w_i]:
            antonym_dict[(antonym1, antonym2)].append((words[w_i], value))
    fig = go.Figure()

    print(antonym_dict)

    # Add axes
    fig.add_shape(type="line", x0=-1, y0=0, x1=1, y1=0, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=-1, y0=0, x1=-1, y1=len(antonym_dict)/10, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=1, y0=0, x1=1, y1=len(antonym_dict)/10, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=0, y0=0, x1=0, y1=len(antonym_dict)/10, line=dict(color='black', width=1, dash='dash'))

    # Define color scale for the words
    colors = np.linspace(0, 1, len(words))

    # Add lines and markers for each word's polarity score
    counter = 0.01
    offset = 0.005
    for i, (antonyms, polars) in enumerate(antonym_dict.items()):
        show_legend = True if i == 0 else False
        x_coords = []
        y_coords = []
        for polar in polars:
            # Define color of the line and marker based on word's position in the list
            color = f'rgb({int(colors[words.index(polar[0])] * 255)}, {int((1-colors[words.index(polar[0])]) * 255)}, 0)'
            x_coords.append(polar[1])
            y_coords.append(counter)
            fig.add_shape(type="line", x0=polar[1], y0=counter, x1=0, y1=counter, 
                          line=dict(color=color, width=2))
            fig.add_trace(go.Scatter(x=[polar[1]], y=[counter], mode='markers', 
                                     marker=dict(color=color, symbol='square', size=10),
                          name=polar[0], showlegend=show_legend))
            # fig.add_annotation(x=polar[1], y=counter, text=polar[0], font=dict(size=20), showarrow=True, xanchor='auto')
        fig.add_annotation(x=-1.1, y=counter, text=antonyms[0], font=dict(size=18), showarrow=False, xanchor='right')
        fig.add_annotation(x=1.1, y=counter, text=antonyms[1], font=dict(size=18), showarrow=False, xanchor='left')
        counter += offset + 0.1
        
    # Join all word markers with lines
    for word, polar in zip(words, polar_dimension):
        x_coords = [p[1] for p in polar]
        y_coords = [counter] * len(polar)
        color = f'rgb({int(colors[words.index(word)] * 255)}, {int((1 - colors[words.index(word)]) * 255)}, 0)'
        fig.add_trace(go.Scatter(x=x_coords, y=y_coords, mode='lines',
                                 line=dict(color=color, width=1),
                                 showlegend=False))
    # Set x and y axis titles
    fig.update_layout(
        xaxis_title=f"Polarity",
        yaxis_title="Words",
        xaxis_range=[-1, 1],
        xaxis_autorange=True, yaxis_autorange=True
    )

    fig.show()


In [107]:
plot_word_polarity([word],[polarity_base_change])

defaultdict(<class 'list'>, {('hot', 'cold'): [('mountain', -0.13891463)], ('intelligent', 'unintelligent'): [('mountain', -0.10718187)], ('bad', 'good'): [('mountain', -0.0989459)], ('capable', 'incapable'): [('mountain', -0.085152745)]})


In [37]:
wp = WordPolarity(model, antonym_path, method="base-change", number_polar=4)

word = "sun"
context = "The sun is shining today."
polarity_base_change = wp.analyze_word(word, context)
    
word1 = "fire"
context1 = "the fire is burning"
polarity_base_change1 = wp.analyze_word(word1, context1)

word2 = "water"
context2 = "the water is super fresh"
polarity_base_change2 = wp.analyze_word(word2, context2)

word3 = "earth"
context3 = "the earth is beautiful"
polarity_base_change3 = wp.analyze_word(word2, context2)

word4 = "video"
context4 = "a video of a supernove"
polarity_base_change4 = wp.analyze_word(word2, context2)

print(polarity_base_change)


Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Top: 1
Dimension: hot <------> cold
['having or showing great eagerness or enthusiasm', 'newly made', 'characterized by violent and forceful activity or movement; very intense', 'newest or most recent', 'very good; often used in the negative', 'sexually excited or exciting', 'very unpleasant or even dangerous', 'producing a burning sensation on the taste nerves', 'wanted by the police', 'extended meanings; especially of psychological heat; marked by intensity or vehemence especially of passion or enthusiasm', 'having or dealing with dangerously high levels of radioactivity', '(color) bold and intense', 'charged or energized with electricity', 'having or bringing unusually good luck', 'performed or performing with unusually great skill and daring and energy', 'marked by excited activity', 'of a seeker; very near to the object sought', 'very popular or successful', 'used of physical heat; having a high or higher than desirable

In [38]:
def plot_polar(word, polar_dimension):
    antonyms = []
    values = []
    for antonym1, antonym2, value in polar_dimension:
        values.append(abs(value))
        if value > 0:
            antonyms.append(antonym1)
        else:
            antonyms.append(antonym2)
    
    antonyms.append(antonyms[0])
    values.append(values[0])
    color = random.choice(px.colors.qualitative.Pastel)

    fig = go.Figure(
        go.Scatterpolar(
        name = word,
        r=values,
        theta=antonyms,
        line_color=color,
        fill="toself",

        )   
    )

    fig.update_layout(
        showlegend = True,
        height = 1000,
        polar=dict(
        radialaxis_angle = 45,
        angularaxis = dict(
            direction ="clockwise",
            period= len(antonyms)-1
            )
        )
    )

    fig.show()
    

In [240]:
import plotly.express as px
import random

print(px.colors.qualitative.Pastel)

['rgb(102, 197, 204)', 'rgb(246, 207, 113)', 'rgb(248, 156, 116)', 'rgb(220, 176, 242)', 'rgb(135, 197, 95)', 'rgb(158, 185, 243)', 'rgb(254, 136, 177)', 'rgb(201, 219, 116)', 'rgb(139, 224, 164)', 'rgb(180, 151, 231)', 'rgb(179, 179, 179)']


In [253]:
plot_polar(word, polarity_base_change)

In [39]:
from plotly.subplots import make_subplots
import math

def plot_word_polarity_polar_fig(words, polar_dimension):
    word_dict = {word: None for word in words}
    for idx, item in enumerate(polar_dimension):
        antonym_dict = {}
        for antonym1, antonym2, value in item:
            if value > 0:
                antonym_dict[antonym1] = value
            else:
                antonym_dict[antonym2] = abs(value)
        word_dict[words[idx]] = antonym_dict

    # print(word_dict)

    num_cols = 2
    num_rows = math.ceil(len(polar_dimension)/num_cols)
    specs = [[{"type":"polar"} for _ in range(num_cols)] for _ in range(num_rows)]

    fig = make_subplots(rows=num_rows, cols=num_cols, specs=specs)

    for idx, word in enumerate(word_dict):
        # print(word_dict[word].keys())
        fig.add_trace(
            go.Scatterpolar(
                r = list(word_dict[word].values()),
                theta = list(word_dict[word].keys()),
                name = word, 
            ), int(idx/2 + 1) , (idx%2 + 1) 
        )

    
    fig.update_traces(fill="toself")
    # fig.update_layout(
    #     polar=dict(
    #     radialaxis_angle = -45,
    #     angularaxis = dict(
    #         direction ="clockwise",
    #         period= len(polar_dimension[0])
    #         )
    #     )
    # )

    fig.show()

        

In [40]:
plot_word_polarity_polar_fig([word, word1],[polarity_base_change, polarity_base_change1])

In [41]:
plot_word_polarity_polar_fig([word, word1, word2, word3, word4],[polarity_base_change, polarity_base_change1, polarity_base_change2, polarity_base_change4, polarity_base_change4])

In [42]:
plot_word_polarity_polar_fig([word],[polarity_base_change])

In [43]:
word1 = "fire"
context1 = "the fire is burning"
wp = WordPolarity(model, antonym_path=antonym_path, method='base-change', number_polar=4)
polarity_base_change1 = wp.analyze_word(word1, context1)

word = "sun"
context = "the sun is shining today and it is a beautiful hot day"
polarity_base_change = wp.analyze_word(word, context)

plot_word_polarity([word, word1],  [polarity_base_change, polarity_base_change1])

Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Top: 1
Dimension: hot <------> cold
['having or showing great eagerness or enthusiasm', 'newly made', 'characterized by violent and forceful activity or movement; very intense', 'newest or most recent', 'very good; often used in the negative', 'sexually excited or exciting', 'very unpleasant or even dangerous', 'producing a burning sensation on the taste nerves', 'wanted by the police', 'extended meanings; especially of psychological heat; marked by intensity or vehemence especially of passion or enthusiasm', 'having or dealing with dangerously high levels of radioactivity', '(color) bold and intense', 'charged or energized with electricity', 'having or bringing unusually good luck', 'performed or performing with unusually great skill and daring and energy', 'marked by excited activity', 'of a seeker; very near to the object sought', 'very popular or successful', 'used of physical heat; having a high or higher than desirable

In [48]:
from collections import defaultdict
import plotly.graph_objects as go

def plot_word_polarity_2d(words, polar_dimension):
    # create dictionary with antonyms as key and (word,polar) as value
    antonym_dict = defaultdict(list)
    for w_i in range(len(words)):
        for antonym1, antonym2, value in polar_dimension[w_i]:
            antonym_dict[(antonym1, antonym2)].append((words[w_i], value))
    
    #create dictionary with word as key and (polar1,polar2) as value
    word_dict = defaultdict(list)
    for w_i in range(len(words)):
        for antonym1, antonym2, value in polar_dimension[w_i]:
            word_dict[words[w_i]].append(value)
    
    fig = go.Figure()

    antonyms = list(antonym_dict.keys())
    # Set axis titles
    fig.update_layout(
        xaxis_range=(-1, 1),
        yaxis_range=(-1, 1)
    )

    # Add lines for each antonym pair
    fig.add_shape(type="line", x0=-1, y0=0, x1=1, y1=0, line=dict(color='black', width=1))
    fig.add_shape(type="line", x0=0, y0=-1, x1=0, y1=1, line=dict(color='black', width=1))

    # Define color scale for the words
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

    # Add markers for each word's polar value
    for word in word_dict:
        x, y = word_dict[word][:2]
        fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers', marker=dict(color=colors.pop(), size=10), name=word))
        
    # Add antonyms as labels on the edges of the plot
    fig.add_annotation(x=-1, y=0, text=antonyms[0][0], showarrow=False, xshift=-15)
    fig.add_annotation(x=1, y=0, text=antonyms[0][1], showarrow=False, xshift=15)
    fig.add_annotation(x=0, y=-1, text=antonyms[1][0], showarrow=False, yshift=-15)
    fig.add_annotation(x=0, y=1, text=antonyms[1][1], showarrow=False, yshift=15)

    fig.show()


In [49]:
wp = WordPolarity(model, antonym_path=antonym_path, method='base-change', number_polar=2)
words = ["earth", "air"]
context = [ "the earth is shaking", "the air is blowing"]

polar_dimensions = []
for word, context in zip(words, context):
    polar_dimensions.append(wp.analyze_word(word, context))

plot_word_polarity_2d(words, polar_dimensions)

Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Top: 1
Dimension: intelligent <------> unintelligent
['exercising or showing good judgment', 'endowed with the capacity to reason', 'possessing sound knowledge', 'having the capacity for thought and reason especially to a high degree'] ['lacking intelligence']
Value: -0.10773206


Top: 2
Dimension: hot <------> cold
['having or showing great eagerness or enthusiasm', 'newly made', 'characterized by violent and forceful activity or movement; very intense', 'newest or most recent', 'very good; often used in the negative', 'sexually excited or exciting', 'very unpleasant or even dangerous', 'producing a burning sensation on the taste nerves', 'wanted by the police', 'extended meanings; especially of psychological heat; marked by intensity or vehemence especially of passion or enthusiasm', 'having or dealing with dangerously high levels of radioactivity', '(color) bold and intense', 'charged or energized with electricity', 'havi

In [50]:
def read_antonym_pairs_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    lines = content.strip().split('\n')
    dimensions = []
    for line in lines:
        split_line = [item.strip() for item in line.split(',')]
        dimensions.append(split_line)
    return dimensions

read_antonym_pairs_file('data/antonym_pairs.csv')



def read_word_context_file(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    lines = content.strip().split('\n')
    words = []
    contexts = []
    for line in lines:
        split_line = [item.strip() for item in line.split(',')]
        words.append(split_line[0])
        contexts.append(split_line[1])
    return words, contexts

read_word_context_file('data/word_context.csv')

[['cold', 'hot'],
 ['bad', 'good'],
 ['intelligent', 'unintelligent'],
 ['capable', 'incapable']]

(['sun',
  'cat',
  'tree',
  'book',
  'car',
  'dog',
  'flower',
  'computer',
  'rain',
  'mountain'],
 ['The sun is shining brightly.',
  'The cat is sleeping on the couch.',
  'There is a tall tree in the park.',
  'I love reading books in my free time.',
  'He drove the car at high speed.',
  'The dog wagged its tail happily.',
  'She picked a beautiful flower from the garden.',
  'I use my computer for work and entertainment.',
  'We need an umbrella when it rains.',
  'They climbed the mountain and enjoyed the view.'])

In [82]:
out_path = './antonyms/'

dimensions = read_antonym_pairs_file('data/antonym_pairs.csv')

dictionary = Dictionary('wordnet', api_key='')    
# dictionary = Dictionary('dictionaryapi', api_key='b4b51989-1b9d-4690-8975-4a83df13efc4 ')
# dictionary = Dictionary('wordnik', api_key='6488daf20061aa3e6200c013b470fa8ef1f2678c19b36ef05')
lookupSpace = LookupCreator(dictionary, dimensions, out_path)
lookupSpace.create_lookup_files()

antonym_path = out_path + "polar_dimensions.pkl"

model = BERTWordEmbeddings()

pdc = PolarDimensions(model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
pdc.create_polar_dimensions(out_path)

words, context = read_word_context_file('data/word_context.csv')
wp = WordPolarity(model, antonym_path=antonym_path, method='base-change', number_polar=3)

polar_dimensions = []
for word, context in zip(words, context):
    dimension = wp.analyze_word(word, context)
    polar_dimensions.append(dimension)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


This word is missing in a corresponding example sentence: bad
Unable to create POLAR dimensions.
Loading antonyms from ./antonyms/lookup_anto_example_dict.pkl
Top: 1
Dimension: cold <------> hot
['having lost freshness through passage of time', 'without compunction or human feeling', 'having a low or inadequate temperature or feeling a sensation of coldness or having been made cold by e.g. ice or refrigeration', 'so intense as to be almost uncontrollable', 'lacking the warmth of life', 'unconscious from a blow or shock or intoxication', 'the absence of heat', 'marked by errorless familiarity', 'extended meanings; especially of psychological coldness; without human warmth or emotion', '(color) giving no sensation of warmth', 'a mild viral infection involving the nose and respiratory passages (but not the lungs)', 'lacking originality or spontaneity; no longer new', 'the sensation produced by low temperatures', 'feeling or showing no enthusiasm', 'sexually unresponsive', 'of a seeker; fa

In [83]:
plot_word_polarity(words,  polar_dimensions)


defaultdict(<class 'list'>, {('cold', 'hot'): [('sun', -0.1621005), ('cat', -0.011349425), ('dog', -0.030429441), ('rain', 0.27137232), ('mountain', -0.026755227)], ('intelligent', 'unintelligent'): [('sun', -0.097381435), ('cat', -0.073004276), ('tree', -0.091014154), ('book', -0.10952695), ('dog', -0.09405656), ('rain', -0.03174801), ('mountain', -0.041489746)], ('bad', 'good'): [('sun', -0.09680711), ('cat', -0.07295258), ('tree', -0.021272952), ('book', -0.060032964), ('dog', -0.042754427), ('rain', 0.023835156), ('mountain', -0.05327501)], ('capable', 'incapable'): [('tree', -0.0073070386), ('book', 0.04213726)]})
